In [ ]:
import pandas as pd

In [ ]:
# Load video IDs from CSV files
train_csv_path = "train.csv"  # Replace with the path to train CSV
eval_csv_path = "eval.csv"    # Replace with the path to eval CSV
test_csv_path = "test.csv"    # Replace with the path to test CSV

train_df = pd.read_csv(train_csv_path)
eval_df = pd.read_csv(eval_csv_path)
test_df = pd.read_csv(test_csv_path)

import json

with open('Transcript.json','r') as f:
    data = json.load(f)


In [ ]:
keys_ids = [k.split('.mp3')[0] for k in data.keys()]

In [ ]:
# data['R-XZUvYcrzE_sep_1105_1440.mp3']['segments']

In [ ]:
# import re

# def split_paragraph(paragraph):
#     # Regular expression to match sentence-ending punctuation that is not part of a number
#     segments = re.split(r'(?<!\d)[.?!]\s+(?=[A-Z])', paragraph)
#     # Remove any empty strings that might occur
#     segments = [segment.strip() for segment in segments if segment.strip()]
#     return segments


In [ ]:
for idx,row in train_df.iterrows():
    if row['video_ids'] =='_H97_Q1CkjE':
        print(row['human_summaries'])

Warren Buffett advises young people to start saving money and accumulating knowledge. He also shares a joke about having a grandson close to a girl's age and advises her to be careful before getting too invested in a relationship.


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:


# def find_matching_segments(inputs, summary):
#         results = []
#     # for input_segments, summary in zip(inputs, summaries):
#         # Compute embeddings for all segments and the summary
#         segment_embeddings = model.encode(inputs, convert_to_tensor=True)
#         summary_embedding = model.encode(summary, convert_to_tensor=True)

#         # Compute cosine similarity
#         similarities = util.cos_sim(summary_embedding, segment_embeddings)

#         # Find the best matching segment(s)
#         best_indices = similarities[0].argsort(descending=True).tolist()

#         # Map indices to scores and segments
#         best_matches = [(i, inputs[i], float(similarities[0][i])) for i in best_indices]
#         results.append(best_matches)
#         return results


# from tqdm.notebook import tqdm

# train_pred ={}


# for idx,row in tqdm(train_df.iterrows()):
#     if row['video_ids'] in keys_ids:
#         inputs = [t['text'] for t in data[row['video_ids']+'.mp3']['segments']]

#         summaries = row['human_summaries']

#         # print(inputs)
#         # print(summaries)
#         # break
#         results = find_matching_segments(inputs, summaries)

#         threshold = 0.3

#         # Filter segment indices with scores greater than the threshold
#         filtered_indices = [index for index, text, score in results[0] if score > threshold]



#         train_pred[row['video_ids']] = filtered_indices
    # print(row['video_ids'])
    # print(filtered_indices)
    # break
    # train_preds[row['video_ids']]={}

# Process inputs and summaries


# # Display results
# for i, matches in enumerate(results):
#     print(f"Input {i+1}:")
#     for index, segment, score in matches:
#         print(f"Segment Index: {index}")
#         print(f"Segment: {segment}")
#         print(f"Score: {score:.4f}")
#         print("-" * 80)


test_pred = {}
eval_pred = {}

# Process test set
for idx, row in tqdm(test_df.iterrows(), desc="Processing Test Set"):
    if row['video_ids'] in keys_ids:
        inputs = [t['text'] for t in data[row['video_ids']+'.mp3']['segments']]

        summaries = row['human_summaries']

        results = find_matching_segments(inputs, summaries)

        threshold = 0.3

        # Filter segment indices with scores greater than the threshold
        filtered_indices = [index for index, text, score in results[0] if score > threshold]

        test_pred[row['video_ids']] = filtered_indices

# # Process eval set
# for idx, row in tqdm(eval_df.iterrows(), desc="Processing Eval Set"):
#     if row['video_ids'] in keys_ids:
#         inputs = [t['text'] for t in data[row['video_ids']+'.mp3']['segments']]


#         summaries = row['human_summaries']
#         results = find_matching_segments(inputs, summaries)

#         threshold = 0.3

#         # Filter segment indices with scores greater than the threshold
#         filtered_indices = [index for index, text, score in results[0] if score > threshold]

#         eval_pred[row['video_ids']] = filtered_indices



Processing Test Set: 0it [00:00, ?it/s]

In [ ]:
# import os
# import shutil

# # Ensure the 'dataset' folder exists
# dataset_root = 'dataset'
# os.makedirs(dataset_root, exist_ok=True)

# # Assuming train_pred is a dictionary with keys as video names and values as segment lists
# for key in train_pred:
#     video_name = key
#     segment = train_pred[key]
#     frame_segment = []

#     # Define the destination directory for this video
#     dest_dir = os.path.join(dataset_root, video_name)

#     # Create the video-specific directory if it doesn't exist
#     os.makedirs(dest_dir, exist_ok=True)

#     # Copy each frame to the destination
#     for val in segment:
#         frame_name = f'segment_{val}.jpg'
#         src_path = os.path.join('source_folder', frame_name)  # Replace 'source_folder' with your actual source folder
#         dest_path = os.path.join(dest_dir, frame_name)

#         # Check if the source file exists before copying
#         if os.path.exists(src_path):
#             shutil.copy(src_path, dest_path)
#         else:
#             print(f"File not found: {src_path}")

#     # Print the copied frames for verification
#     print(f"Copied frames for video '{video_name}' to {dest_dir}")
#     break  # Remove this break if you want to process all keys


In [ ]:
import os
import shutil

# Define dataset root and ensure it exists
dataset_root = 'dataset'
os.makedirs(dataset_root, exist_ok=True)

# Function to copy frames for a specific dataset type
def copy_frames(subset_name, subset_pred):
    subset_dir = os.path.join(dataset_root, subset_name)
    os.makedirs(subset_dir, exist_ok=True)  # Ensure the subset folder exists

    for key in subset_pred:
        video_name = key
        segment = subset_pred[key]

        # Define the destination directory for this video
        dest_dir = os.path.join(subset_dir, video_name)
        if os.path.exists(dest_dir):
            print(f"Directory '{dest_dir}' already exists. Skipping...")
            continue  # Skip if the directory already exists
        os.makedirs(dest_dir, exist_ok=True)  # Create video-specific directory

        # Copy each frame to the destination
        for val in segment:
            frame_index = val
            while True:
                try:
                    frame_name = f'segment_{frame_index}.jpg'
                    src_path = os.path.join(f'data_frames/{key}/', frame_name)  # Replace 'source_folder' with actual source folder
                    dest_path = os.path.join(dest_dir, frame_name)

                    # Check if frame already exists in the destination
                    if os.path.exists(dest_path):
                        print(f"Frame '{dest_path}' already exists. Skipping...")
                        break

                    # Check if the source file exists before copying
                    if os.path.exists(src_path):
                        shutil.copy(src_path, dest_path)
                        print(f"Copied: {src_path} -> {dest_path}")
                        break
                    else:
                        print(f"File not found: {src_path}. Trying next frame.")
                        frame_index += 1

                        # Prevent infinite loop by limiting frame index
                        if frame_index > 1000:
                            print(f"No frames found for {frame_name} after searching.")
                            break
                except Exception as e:
                    print(f"Error copying frame '{frame_name}': {e}")
                    break

# Example dictionaries for train, test, and eval
# train_pred = {
#     "video1": [1, 2, 3],
#     "video2": [4, 5],
# }

# test_pred = {
#     "video3": [1, 2],
# }

# eval_pred = {
#     "video4": [1, 2, 3, 4],
# }

# Process train, test, and eval datasets
#copy_frames("train", train_pred)
copy_frames("test", test_pred)
#copy_frames("eval", eval_pred)


Copied: data_frames/ttJlbPchk5Q/segment_9.jpg -> dataset/test/ttJlbPchk5Q/segment_9.jpg
Copied: data_frames/ttJlbPchk5Q/segment_3.jpg -> dataset/test/ttJlbPchk5Q/segment_3.jpg
Copied: data_frames/ttJlbPchk5Q/segment_7.jpg -> dataset/test/ttJlbPchk5Q/segment_7.jpg
Copied: data_frames/ttJlbPchk5Q/segment_8.jpg -> dataset/test/ttJlbPchk5Q/segment_8.jpg
Copied: data_frames/ttJlbPchk5Q/segment_1.jpg -> dataset/test/ttJlbPchk5Q/segment_1.jpg
Copied: data_frames/ttJlbPchk5Q/segment_10.jpg -> dataset/test/ttJlbPchk5Q/segment_10.jpg
Copied: data_frames/ttJlbPchk5Q/segment_14.jpg -> dataset/test/ttJlbPchk5Q/segment_14.jpg
Copied: data_frames/ttJlbPchk5Q/segment_4.jpg -> dataset/test/ttJlbPchk5Q/segment_4.jpg
Copied: data_frames/ttJlbPchk5Q/segment_2.jpg -> dataset/test/ttJlbPchk5Q/segment_2.jpg
Copied: data_frames/ttJlbPchk5Q/segment_16.jpg -> dataset/test/ttJlbPchk5Q/segment_16.jpg
Copied: data_frames/ttJlbPchk5Q/segment_11.jpg -> dataset/test/ttJlbPchk5Q/segment_11.jpg
File not found: data_fra

In [ ]:
# train_pred.keys()

In [ ]:
# import os
# len(os.listdir('dataset/test'))